# 1.Imports

In [11]:
import torch
import torchvision
import torch.nn as nn
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, random_split
import numpy as np

# 2.Data 

## 2.1 Loaders

### 2.1.1 MNIST Data

In [3]:
transform = transforms.Compose([transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,)),
    ])
batch_size = 64

datasets_save_dir = 'Data/'

train_data = datasets.MNIST(root=datasets_save_dir,download=True,train=True, 
    transform=transform)
test_data = datasets.MNIST(root=datasets_save_dir,download=True,train=False, 
    transform=transform)

train_dataloader = DataLoader(train_data,batch_size=batch_size,
    shuffle=True)
test_dataloader = DataLoader(test_data,batch_size=batch_size,shuffle=True)

### 2.1.2 Subsets

In [4]:

subset_sizes = [1000*i for i in range(1,6)] 

subset_loaders = []
for subset_size in subset_sizes:
    subset, _ = random_split(train_data, [subset_size, len(train_data) - subset_size])
    subset_loader = DataLoader(subset, batch_size=batch_size, shuffle=True)
    subset_loaders.append(subset_loader)


## 2.2 Data shape

In [5]:
for images,labels in train_dataloader:
    print(images.shape)
    print(labels.shape)
    break

torch.Size([64, 1, 28, 28])
torch.Size([64])


## 3.Model

In [6]:
class Fcnn(nn.Module):
    def __init__(self):
        super().__init__()

        self.block_1 = nn.Sequential(
            nn.Linear(28*28,256),
            nn.ReLU(),
            nn.Linear(256,64),
            nn.ReLU(),
            nn.Linear(64,10),
            nn.Softmax()
        )

    def forward(self,x):
        x = self.block_1(x)
        return x

# 4.Training

In [7]:
def train(epochs:int,model,optimizer,criterion,data):
    model.train()
    loss_history = []
    for epoch in range(epochs):
        for batch, (images,labels) in enumerate(data):
            images = images.view(images.shape[0],-1)

            optimizer.zero_grad()
            
            output = model(images)

            loss = criterion(output,labels) 
            loss.backward()

            optimizer.step()

            if (batch+1) % 100 == 0:
                print(f'epoch: {epoch+1}, step: {batch+1}/{len(data)} , loss: {loss.item():.4f}')
        loss_history.append(loss.item())
            
    print('Finished training!')
    return loss_history

In [53]:
model = Fcnn()
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

train(3,model,optimizer,criterion,train_dataloader)

epoch: 1, step: 100/3750 , loss: 1.8908
epoch: 1, step: 200/3750 , loss: 1.7894
epoch: 1, step: 300/3750 , loss: 1.5904
epoch: 1, step: 400/3750 , loss: 1.7767
epoch: 1, step: 500/3750 , loss: 1.6523
epoch: 1, step: 600/3750 , loss: 1.6694
epoch: 1, step: 700/3750 , loss: 1.6568
epoch: 1, step: 800/3750 , loss: 1.7092
epoch: 1, step: 900/3750 , loss: 1.5276
epoch: 1, step: 1000/3750 , loss: 1.6782
epoch: 1, step: 1100/3750 , loss: 1.5820
epoch: 1, step: 1200/3750 , loss: 1.5494
epoch: 1, step: 1300/3750 , loss: 1.4786
epoch: 1, step: 1400/3750 , loss: 1.5354
epoch: 1, step: 1500/3750 , loss: 1.5785
epoch: 1, step: 1600/3750 , loss: 1.5732
epoch: 1, step: 1700/3750 , loss: 1.6300
epoch: 1, step: 1800/3750 , loss: 1.5348
epoch: 1, step: 1900/3750 , loss: 1.6057
epoch: 1, step: 2000/3750 , loss: 1.5254
epoch: 1, step: 2100/3750 , loss: 1.5164
epoch: 1, step: 2200/3750 , loss: 1.5454
epoch: 1, step: 2300/3750 , loss: 1.5258
epoch: 1, step: 2400/3750 , loss: 1.5317
epoch: 1, step: 2500/3750

## 5.Testing

In [8]:
def test(model, data):
    model.eval()
    correct = 0
    with torch.no_grad():
        for batch, (images,labels) in enumerate(data):
            images = images.view(images.shape[0],-1)

            output = model(images)

            pred = output.data.max(1,keepdim=True)[1]
            
            labels = labels.data.view_as(pred)
            correct += pred.eq(labels).sum()
    accuracy = correct/len(data.dataset)
    print(f'Accuracy: {accuracy:.3f}')
    return accuracy


In [14]:

test(model)
        

Accuracy: 0.970


## 6. Early stopping

In [24]:
class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = float('inf')

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False
    
def train_es(epochs:int,model,optimizer,criterion,dataloader):
    early_stopper = EarlyStopper(patience=2, min_delta=10)
    model.train()
    for epoch in range(epochs):
        batch_loss_history = []
        for batch, (images,labels) in enumerate(dataloader):
            images = images.view(images.shape[0],-1)

            optimizer.zero_grad()
            
            output = model(images)

            loss = criterion(output,labels)
            loss.backward()

            optimizer.step()
            if (batch+1) % 100 == 0:
                print(f'epoch: {epoch+1}, step: {batch+1}/{len(dataloader)} , loss: {loss.item():.4f}')
            batch_loss_history.append(loss.item().numpy())
        
        print(torch.float)
        epoch_avg_loss = np.average(batch_loss_history)
        if early_stopper.early_stop(epoch_avg_loss):
            print('Early stopping!')
            break
        
        print(f'Epoch: {epoch+1}, avg_loss = {epoch_avg_loss}')
    print('Finished training!')



In [25]:
model = Fcnn()
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

train_es(20,model,optimizer,criterion,dataloader = subset_loaders[0])

/home/luki/Dokumenty/Projekty/sieci_projekt2/venv/lib64/python3.11/site-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


AttributeError: 'float' object has no attribute 'numpy'

## 7. Training on different subsets

In [44]:
criterion = nn.CrossEntropyLoss()

subset_loss_history = []
subset_accuracy_train = []
subset_accuracy_test = []
for subset in subset_loaders:
    model = Fcnn()
    optimizer = torch.optim.Adam(model.parameters())
    loss_history = train(3,model,optimizer,criterion,subset)
    subset_loss_history.append(loss_history)

    accuracy_test = test(model,test_dataloader)
    accuracy_train = test(model,train_dataloader)
    
    subset_accuracy_train.append(accuracy_train)
    subset_accuracy_test.append(accuracy_test)

/home/luki/Dokumenty/Projekty/sieci_projekt2/venv/lib64/python3.11/site-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


Finished training!
Accuracy: 0.685
Accuracy: 0.683
Finished training!
Accuracy: 0.791
Accuracy: 0.780
Finished training!
Accuracy: 0.719
Accuracy: 0.710
epoch: 1, step: 100/125 , loss: 1.8584
epoch: 2, step: 100/125 , loss: 1.6340
epoch: 3, step: 100/125 , loss: 1.6458
Finished training!
Accuracy: 0.818
Accuracy: 0.812
epoch: 1, step: 100/157 , loss: 1.7036
epoch: 2, step: 100/157 , loss: 1.7441
epoch: 3, step: 100/157 , loss: 1.5964
Finished training!
Accuracy: 0.840
Accuracy: 0.843


In [2]:
plt.plot(subset_sizes,subset_accuracy_train, label='Train accuracy')
plt.plot(subset_sizes,subset_accuracy_test ,label='Test accuracy')


NameError: name 'plt' is not defined